In [1]:
%pip install psycopg2
%pip install python-dotenv
%pip install folium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
%matplotlib inline
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
load_dotenv('credentials.env')
print("DB_HOST:", os.getenv("DB_HOST"))
print("DB_NAME:", os.getenv("DB_NAME"))
print("DB_USER:", os.getenv("DB_USER"))
print("DB_PASSWORD:", os.getenv("DB_PASSWORD"))
print("DB_PORT:", os.getenv("DB_PORT"))

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT"),
)
cur = conn.cursor()

DB_HOST: pinot.cs.ucsb.edu
DB_NAME: pinot
DB_USER: cs190nf24g10
DB_PASSWORD: YKUkr3GV
DB_PORT: 5432


# Wifi Stats Data Preprocessing

In [5]:
query = "SELECT * FROM public.wifistats"
cur.execute(query)
data = cur.fetchall()

In [7]:
wifi_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "time",
        "ping_str",
        "signal (dBm)",
        "tx_bitrate_str",
        "rx_bitrate_str",
        "connected_time_str",
        "packet_loss",
    ],
)

wifi_df["connected_time (s)"] = (
    wifi_df["connected_time_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["tx_bitrate (MBit/s)"] = (
    wifi_df["tx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["rx_bitrate (MBit/s)"] = (
    wifi_df["rx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)

pattern = r"min/avg/max/mdev = ([\d\.]+)/([\d\.]+)/([\d\.]+)/([\d\.]+) ms"
extracted_values = wifi_df["ping_str"].str.extract(pattern).astype(float)
extracted_values.columns = [
    "min_ping_rtt (ms)",
    "avg_ping_rtt (ms)",
    "max_ping_rtt (ms)",
    "mdev_ping_rtt (ms)",
]

wifi_df = pd.concat([wifi_df, extracted_values], axis=1)
wifi_df = wifi_df.drop(
    ["connected_time_str", "tx_bitrate_str", "rx_bitrate_str", "ping_str"], axis=1
)

wifi_df.to_csv("./data/wifistats.csv", index=False)

display(wifi_df.head())

,label,time,signal (dBm),packet_loss,connected_time (s),tx_bitrate (MBit/s),rx_bitrate (MBit/s),min_ping_rtt (ms),avg_ping_rtt (ms),max_ping_rtt (ms),mdev_ping_rtt (ms)
0,raspi-e4:5f:01:84:8c:99,2024-02-13 00:46:21.750007,-65.0,0.0,546569.0,200.0,150.0,3.308,3.878,4.840,0.572
1,raspi-e4:5f:01:a0:4a:e3,2024-02-13 00:46:21.750573,-72.0,0.0,589099.0,90.0,45.0,2.852,5.245,12.919,3.893
2,raspi-e4:5f:01:a0:1e:bb,2024-02-13 00:46:21.751007,-63.0,0.0,5744.0,72.2,65.0,3.071,9.624,23.856,7.748
3,raspi-e4:5f:01:8c:e3:3f,2024-02-13 00:46:21.751420,-62.0,0.0,772224.0,150.0,150.0,2.987,4.495,9.138,2.375
4,raspi-e4:5f:01:84:80:6f,2024-02-13 00:46:21.751828,-57.0,0.0,522852.0,180.0,200.0,2.737,4.442,9.262,2.453


# Device Data Preprocessing

In [8]:
query = "SELECT * FROM public.devices"
cur.execute(query)
data = cur.fetchall()

In [9]:
devices_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "ethernet_mac",
        "wlan_mac",
        "location",
        "tags",
        "comment",
        "uptime",
        "wireless_bytes",
        "wired_bytes",
        "last_seen",
    ],
)
devices_df.to_csv("./data/devices.csv", index=False)
display(devices_df.head())

,label,ethernet_mac,wlan_mac,location,tags,comment,uptime,wireless_bytes,wired_bytes,last_seen
0,raspi-dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:65,None,None,None,"18:11:40 up 206 days, 17:13, 0 users, load ...",7.723500e+07,1.246234e+10,2023-11-15 18:11:55.004674
1,raspi-e4:5f:01:72:89:99,e4:5f:01:72:89:99,e4:5f:01:72:89:9a,None,None,None,"17:43:43 up 154 days, 23:15, 0 users, load ...",5.499737e+07,7.350356e+09,2023-09-27 17:43:58.456155
2,raspi-e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e6,None,None,None,None,NaN,NaN,NaT
3,raspi-e4:5f:01:56:d6:ce,e4:5f:01:56:d6:ce,e4:5f:01:56:d6:cf,None,None,None,"18:11:40 up 206 days, 17:06, 2 users, load ...",1.672108e+09,1.082823e+10,2023-11-15 18:11:55.017829
4,raspi-e4:5f:01:75:6e:53,e4:5f:01:75:6e:53,e4:5f:01:75:6e:54,None,None,None,"18:11:40 up 203 days, 23:40, 0 users, load ...",6.836665e+07,1.446518e+10,2023-11-15 18:11:55.003112


# Extra Info Data Preprocessing

In [10]:
query = "SELECT * FROM public.extra_info"
cur.execute(query)
data = cur.fetchall()

In [11]:
extra_info_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "comment",
        "contact_person",
        "latitude",
        "longitude",
    ],
)
extra_info_df["latitude"].replace("None", np.nan, inplace=True)
extra_info_df["longitude"].replace("None", np.nan, inplace=True)
extra_info_df = extra_info_df.dropna(subset=["latitude", "longitude"]).reset_index(
    drop=True
)
extra_info_df = extra_info_df.drop(["comment", "contact_person"], axis=1)
extra_info_df.to_csv("./data/extra_info.csv", index=False)
display(extra_info_df.head())

,label,latitude,longitude
0,raspi-e4:5f:01:75:6b:2c,34.41773457866324,-119.8546902810392
1,raspi-e4:5f:01:8e:27:aa,34.409574095820076,-119.85180308648167
2,raspi-e4:5f:01:a7:b2:7e,34.409548306773345,-119.85153880818854
3,raspi-e4:5f:01:75:54:04,34.418302607166105,-119.85669070847874
4,raspi-e4:5f:01:9b:84:c4,34.408679667265304,-119.85184124971644
